In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
DATAPATH = './data/games.csv'

In [3]:
games = pd.read_csv(DATAPATH)

In [4]:
games.dtypes

Unnamed: 0             int64
SEASON_ID              int64
TEAM_ID                int64
TEAM_ABBREVIATION     object
TEAM_NAME             object
GAME_ID                int64
GAME_DATE             object
MATCHUP               object
WL                    object
MIN                    int64
PTS                    int64
FGM                    int64
FGA                    int64
FG_PCT               float64
FG3M                   int64
FG3A                   int64
FG3_PCT              float64
FTM                    int64
FTA                    int64
FT_PCT               float64
OREB                   int64
DREB                   int64
REB                    int64
AST                    int64
STL                    int64
BLK                    int64
TOV                    int64
PF                     int64
PLUS_MINUS           float64
dtype: object

In [5]:
games = games.drop(games.columns[0], axis=1)

In [6]:
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22019,1610612748,MIA,Miami Heat,21900970,2020-03-11,MIA vs. CHA,L,239,98,...,0.833,9,27,36,32,8,5,14,17,-13.0
1,22019,1610612743,DEN,Denver Nuggets,21900973,2020-03-11,DEN @ DAL,L,240,97,...,0.625,3,37,40,23,6,4,15,20,-7.8
2,22019,1610612752,NYK,New York Knicks,21900969,2020-03-11,NYK @ ATL,W,264,136,...,0.735,6,41,47,32,14,9,12,23,0.0
3,22019,1610612742,DAL,Dallas Mavericks,21900973,2020-03-11,DAL vs. DEN,W,240,113,...,0.739,13,39,52,23,9,2,10,14,8.0
4,22019,1610612765,DET,Detroit Pistons,21900971,2020-03-11,DET @ PHI,L,241,106,...,0.680,6,26,32,25,10,3,8,24,-14.2


In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('WL')
    ds = tf.data.Dataset.from_tensor_slices((dataframe.values, labels.values))
    if shuffle:
        ds = ds.shuffle(buffer_size = len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [8]:
games.pop('SEASON_ID')
games.pop('TEAM_ID')
games.pop('GAME_ID')

0        21900970
1        21900973
2        21900969
3        21900973
4        21900971
           ...   
29995    40800221
29996    40800211
29997    40800231
29998    40800137
29999    40800137
Name: GAME_ID, Length: 30000, dtype: int64

In [9]:
feature_columns = []
obj_columns = ['TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_DATE', 'MATCHUP', 'WL']
#numeric columns
for i in obj_columns:
    games[i] = pd.Categorical(games[i])
    games[i] = games[i].cat.codes 
for header in games.columns:
    games[header] = games[header].astype('float64')
    if header != 'WL':
        feature_columns.append(feature_column.numeric_column(header))

In [10]:
games.dtypes

TEAM_ABBREVIATION    float64
TEAM_NAME            float64
GAME_DATE            float64
MATCHUP              float64
WL                   float64
MIN                  float64
PTS                  float64
FGM                  float64
FGA                  float64
FG_PCT               float64
FG3M                 float64
FG3A                 float64
FG3_PCT              float64
FTM                  float64
FTA                  float64
FT_PCT               float64
OREB                 float64
DREB                 float64
REB                  float64
AST                  float64
STL                  float64
BLK                  float64
TOV                  float64
PF                   float64
PLUS_MINUS           float64
dtype: object

In [11]:
print(feature_columns)

[NumericColumn(key='TEAM_ABBREVIATION', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='TEAM_NAME', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='GAME_DATE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MATCHUP', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='MIN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FGA', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG_PCT', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FG3M', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), Numer

In [12]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [13]:
train, test = train_test_split(games, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

19200 train examples
4800 validation examples
6000 test examples


In [14]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


In [15]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [17]:
model = get_compiled_model()
model.fit(train_ds, epochs=30)

Train for 600 steps
Epoch 1/30
600/600 [==============================] - 3s 4ms/step - loss: nan - accuracy: 0.4983
Epoch 2/30
600/600 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.4978
Epoch 3/30
600/600 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.4978
Epoch 4/30
600/600 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.4978
Epoch 5/30
600/600 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.4978
Epoch 6/30
600/600 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.4978
Epoch 7/30
600/600 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.4978
Epoch 8/30
600/600 [==============================] - 2s 4ms/step - loss: nan - accuracy: 0.4978
Epoch 9/30
600/600 [==============================] - 3s 5ms/step - loss: nan - accuracy: 0.4978
Epoch 10/30
600/600 [==============================] - 3s 5ms/step - loss: nan - accuracy: 0.4978
Epoch 11/